# Otimização dos Parâmetros 


Todo modelo de Machine Learning possui parâmetros que permitem a customização do modelo. Esses parâmetros também são chamados de hiperparâmetros.

Em programação os algoritmos de Machine Learning são representados por funções e cada função possui os parâmetros de customização, exatamente o que chamamos de hiperparâmetros.

É comum ainda que as pessoas se refiram aos coeficientes do modelo (encontrados ao final do treinamento) como parâmetros.

Parte do nosso trabalho como Cientistas de Dados é encontrar a melhor combinação de hiperparâmetros para cada modelo.

Em Métodos Ensemble esse trabalho é ainda mais complexo, pois temos os hiperparâmetros do estimador base e os hiperparâmetros do modelo ensemble, conforme este exemplo abaixo:

* Estimador base:

estim_base = KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski', metric_params=None, n_jobs=None, n_neighbors=5, p=2, weights='uniform')

* Modelo Ensemble:

BaggingClassifier(base_estimator=estim_base,
                  bootstrap=True, bootstrap_features=False, max_features=0.5,
                  max_samples=0.5, n_estimators=10, n_jobs=None,
                  oob_score=False, random_state=None, verbose=0,
                  warm_start=False)

## Extremely Randomized Forest

Modelo padrão, com hiperparâmetros escolhidos manualmente.

## Importando Bibliotecas

In [1]:

import pandas as pd
import numpy as np
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import scale
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

## Carrega o dataset

In [2]:

data = pd.read_excel('credit.xls', skiprows = 1)

In [3]:
print(data)

          ID  LIMIT_BAL  SEX  EDUCATION  MARRIAGE  AGE  PAY_0  PAY_2  PAY_3  \
0          1      20000    2          2         1   24      2      2     -1   
1          2     120000    2          2         2   26     -1      2      0   
2          3      90000    2          2         2   34      0      0      0   
3          4      50000    2          2         1   37      0      0      0   
4          5      50000    1          2         1   57     -1      0     -1   
...      ...        ...  ...        ...       ...  ...    ...    ...    ...   
29995  29996     220000    1          3         1   39      0      0      0   
29996  29997     150000    1          3         2   43     -1     -1     -1   
29997  29998      30000    1          2         2   37      4      3      2   
29998  29999      80000    1          3         1   41      1     -1      0   
29999  30000      50000    1          2         1   46      0      0      0   

       PAY_4  ...  BILL_AMT4  BILL_AMT5  BILL_AMT6 

In [4]:
target = 'default payment next month'
y = np.asarray(data[target])

In [5]:
features = data.columns.drop(['ID', target])
X = np.asarray(data[features])

## Divisão de dados de treino e de teste

In [6]:

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.30, random_state = 99)

## Classificador

In [7]:

clf = ExtraTreesClassifier(n_estimators = 500, random_state = 99)

In [8]:

clf.fit(X_train, y_train)

ExtraTreesClassifier(n_estimators=500, random_state=99)

In [9]:

scores = cross_val_score(clf, X_train, y_train, cv = 3, scoring = 'accuracy', n_jobs = -1)

In [10]:

print ("ExtraTreesClassifier -> Acurácia em Treino: Média = %0.3f Desvio Padrão = %0.3f" % (np.mean(scores), np.std(scores)))

ExtraTreesClassifier -> Acurácia em Treino: Média = 0.812 Desvio Padrão = 0.002


### Fazendo previsões

In [11]:

y_pred = clf.predict(X_test)

In [12]:

confusionMatrix = confusion_matrix(y_test, y_pred)
print (confusionMatrix)

[[6532  446]
 [1273  749]]


In [13]:

print("Acurácia em Teste:", accuracy_score(y_test, y_pred))

Acurácia em Teste: 0.809


## Otimização dos Hiperparâmetros com Randomized Search

http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.RandomizedSearchCV.html

O Randomized Search gera amostras dos parâmetros dos algoritmos a partir de uma distribuição randômica uniforme para um número fixo de interações. Um modelo é construído e testado para cada combinação de parâmetros. 

In [14]:

from sklearn.model_selection import RandomizedSearchCV

## Definição dos parâmetros

In [15]:

param_dist = {"max_depth": [1, 3, 7, 8, 12, None],
              "max_features": [8, 9, 10, 11, 16, 22],
              "min_samples_split": [8, 10, 11, 14, 16, 19],
              "min_samples_leaf": [1, 2, 3, 4, 5, 6, 7],
              "bootstrap": [True, False]}

# Para o classificador criado com ExtraTrees, testamos diferentes combinações de parâmetros
rsearch = RandomizedSearchCV(clf, param_distributions = param_dist, n_iter = 25, return_train_score = True)  

# Aplicando o resultado ao conjunto de dados de treino e obtendo o score
rsearch.fit(X_train, y_train)

# Resultados 
rsearch.cv_results_

# Imprimindo o melhor estimador
bestclf = rsearch.best_estimator_
print (bestclf)

# Aplicando o melhor estimador para realizar as previsões
y_pred = bestclf.predict(X_test)

# Confusion Matrix
confusionMatrix = confusion_matrix(y_test, y_pred)
print(confusionMatrix)

# Acurácia
accuracy = accuracy_score(y_test, y_pred)
print(accuracy)

ExtraTreesClassifier(bootstrap=True, max_depth=8, max_features=22,
                     min_samples_split=8, n_estimators=500, random_state=99)
[[6654  324]
 [1288  734]]
0.8208888888888889


In [16]:
# Obtendo o grid com todas as combinações de parâmetros
rsearch.cv_results_

{'mean_fit_time': array([ 2.22649941,  1.44481773,  3.28481369,  3.43068876, 10.31056285,
         0.86719351,  5.21497374,  0.87550306,  5.15740538,  1.50294113,
         0.88331637,  2.69869266,  1.03824015,  3.23613348,  5.3436697 ,
         2.11492491,  4.0235198 ,  4.50958056,  6.21740041,  4.46645818,
         1.46153326,  3.54869757,  0.88855495,  3.93384643,  4.00632968]),
 'std_fit_time': array([0.04015654, 0.02999644, 0.02640765, 0.01775721, 0.17537885,
        0.00846861, 0.04836519, 0.008433  , 0.04404279, 0.03536134,
        0.01796736, 0.0887188 , 0.00533299, 0.02231633, 0.06742864,
        0.01871818, 0.01426788, 0.05675014, 0.12329676, 0.17506207,
        0.01930148, 0.02970784, 0.00736581, 0.02358259, 0.02162378]),
 'mean_score_time': array([0.09622593, 0.09561696, 0.13422356, 0.1442749 , 0.30021358,
        0.09717011, 0.27620621, 0.08340411, 0.30271602, 0.09628086,
        0.08346658, 0.14623241, 0.08467288, 0.13517261, 0.13476357,
        0.1305181 , 0.13507218, 0.1

## Grid Search x Randomized Search para Estimação dos Hiperparâmetros

http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html

http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.RandomizedSearchCV.html

O Grid Search realiza metodicamente combinações entre todos os parâmetros do algoritmo, criando um grid. 

In [17]:
import numpy as np
from time import time
from scipy.stats import randint as sp_randint
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.datasets import load_digits

# Obtém o dataset
digits = load_digits()
X, y = digits.data, digits.target

# Construindo o classificador
clf = RandomForestClassifier(n_estimators = 20)

In [18]:
# Randomized Search

# Valores dos parâmetros que serão testados
param_dist = {"max_depth": [3, None],
              "max_features": sp_randint(1, 11),
              "min_samples_leaf": sp_randint(1, 11),
              "bootstrap": [True, False],
              "criterion": ["gini", "entropy"]}

# Executando o Randomized Search
n_iter_search = 20
random_search = RandomizedSearchCV(clf, 
                                   param_distributions = param_dist, 
                                   n_iter = n_iter_search,
                                   return_train_score=True)

start = time()
random_search.fit(X, y)
print("RandomizedSearchCV executou em %.2f segundos para %d candidatos a parâmetros do modelo." 
      % ((time() - start), n_iter_search))

# Imprime as combinações dos parâmetros e suas respectivas médias de acurácia
random_search.cv_results_

RandomizedSearchCV executou em 3.65 segundos para 20 candidatos a parâmetros do modelo.


{'mean_fit_time': array([0.04154191, 0.01782279, 0.03387337, 0.02235541, 0.02575979,
        0.02737598, 0.04047785, 0.02603755, 0.02817826, 0.05535321,
        0.01975117, 0.01457906, 0.01842909, 0.0223628 , 0.02487974,
        0.02273412, 0.02402091, 0.02204661, 0.03973656, 0.05146437]),
 'std_fit_time': array([0.00054871, 0.00011562, 0.00084514, 0.00028159, 0.00038912,
        0.00067358, 0.00014743, 0.00026471, 0.00035085, 0.00047914,
        0.00016299, 0.00012065, 0.00027589, 0.00027496, 0.00019999,
        0.00044516, 0.00031945, 0.00016706, 0.00022116, 0.00052423]),
 'mean_score_time': array([0.00249333, 0.00225134, 0.00265765, 0.00247235, 0.0022644 ,
        0.00267935, 0.00255375, 0.0023385 , 0.00258951, 0.00257564,
        0.00223522, 0.00225453, 0.00224972, 0.00244975, 0.00227218,
        0.00233817, 0.00272698, 0.00222154, 0.00259738, 0.0026257 ]),
 'std_score_time': array([2.37708943e-05, 1.48244984e-05, 9.03428370e-05, 1.08924721e-04,
        3.71439313e-05, 1.27629454e-

In [19]:
# Grid Search

# Usando um grid completo de todos os parâmetros
param_grid = {"max_depth": [3, None],
              "max_features": [1, 3, 10],
              "min_samples_leaf": [1, 3, 10],
              "bootstrap": [True, False],
              "criterion": ["gini", "entropy"]}

# Executando o Grid Search
grid_search = GridSearchCV(clf, param_grid = param_grid, return_train_score = True)
start = time()
grid_search.fit(X, y)

print("GridSearchCV executou em %.2f segundos para todas as combinações de candidatos a parâmetros do modelo."
      % (time() - start))
grid_search.cv_results_

GridSearchCV executou em 12.30 segundos para todas as combinações de candidatos a parâmetros do modelo.


{'mean_fit_time': array([0.01893167, 0.01856918, 0.01842432, 0.02028866, 0.02028203,
        0.02013059, 0.02574759, 0.02581701, 0.02565928, 0.02529974,
        0.02136226, 0.0199616 , 0.0289711 , 0.02726946, 0.02445111,
        0.04410868, 0.04120412, 0.03673239, 0.01891665, 0.01852374,
        0.01842227, 0.02073135, 0.02063732, 0.02072692, 0.02754741,
        0.02764101, 0.02782211, 0.02807994, 0.02283974, 0.02045412,
        0.03317943, 0.0302629 , 0.02605796, 0.05202932, 0.04833641,
        0.04144273, 0.01484036, 0.01459241, 0.01448984, 0.01701188,
        0.01704059, 0.01718402, 0.02548418, 0.02578802, 0.02543592,
        0.02592907, 0.01977177, 0.01692472, 0.03127561, 0.02824831,
        0.02466483, 0.0549634 , 0.05092616, 0.04507217, 0.01486101,
        0.01473432, 0.01459694, 0.01782894, 0.01773005, 0.01763248,
        0.02787571, 0.02825475, 0.02796907, 0.02894959, 0.02170382,
        0.01761055, 0.03669543, 0.03265343, 0.02726054, 0.06446705,
        0.060079  , 0.05145216]